In [26]:
import pandas as pd
import re

## Основные методы

In [24]:
def tokenize(string, comma=False):
  for stopword in {r'\n', r'\r'}:
    string = string.replace(stopword, '')
  string = re.sub(r"[\d]+", ' \g<0> ', string)
  if not comma:
    return re.findall(r'\w+(?:-\w+)+|[\w]+', str.lower(string))
  else:
    return re.findall(r'\w+(?:-\w+)+|[\w]+|\,', str.lower(string)) 

In [84]:
def diff_sets(set1, set2, silent = False):
  '''
  Удаляет из set1 значения set2
  Возвращает что осталось от set1
  '''
  set3 = set1.copy()
  n = 0
  for value in set1:
    if value in set2:
      set3.remove(value)
      n+=1
  if not silent:
    print('Удалено {0} записей'.format(n))
  return set3

def whole_set(set_name):
  if isinstance(set_name, set) or isinstance(set_name, list):
    return set_name.copy()
  elif isinstance(set_name, dict):
    result = {}
    for key, value in set_name.items():
      try:
        result.update(whole_set(value))
      except ValueError:
        result = whole_set(value)
    return result
  
def remove(array, element):
  '''
  Удаляет элемент из массива если он там существует. Не понятно почему родной метод работает по-другому.
  '''
  if element in array:
    array.remove(element)

## Методы для загрузки словаря

In [333]:
def load_types():
  sep_region_signs = {
    'обл.': {"область", "обл", "обл-ть", 'МО'},
    'респ.': {"республика", 'респ'},
    'край': {'край'},
    'город': {'г', 'гор', 'город'},
    'АО':{'ао', 'автономный', 'округ', 'автономная'}
  }

  sep_street_signs = {
    'аллея':{'аллея', 'а'},
    'бульвар':{'б-р', 'бульвар', 'б'}, #- надо добавлять special case
    'наб.':{'наб', 'набережная'},
    'переулок':{'пер', 'переулок'},
    'площадь':{'пл', "площадь"},
    "проспект":{"пр", "пр-кт", "просп"},
    "проезд":{"проезд", "пр-д"},
    "ул.":{"улица", "ул", "у", 'ул-ца'}
  }

  sep_district_signs = {
    'р-н':{'район', "р" , "р-н"},
    'п':{'п', 'пгт', 'поселок', "посёлок", "городского", "типа"}
  }

  sep_city_signs  = {
    'город':{'г', 'гор', 'город'},
    'село':{'с', 'село', 'сел'},
    'деревня':{'д', 'дер', 'деревня', 'д-ня'},
    'с/п':{"сельский", "поселок", "сп"}
  }

  sep_house_signs = {
    'дом': {'вл', 'д', 'дом'},
    'корпус': {'к', 'корп', 'копр', 'кор', 'корпус', 'с', 'стр', 'строен', 'строение'},
    'квартира': {'кабинет', 'кв', 'квартира', "ком",'комн', 'пом', 'помещение', 'комната'},
    'офис': {'оф', 'офис'},
    'литера': {'а', 'б', 'в'}
  }
  
  types = {
    'город':sep_city_signs,
    'дом':sep_house_signs,
    'улица':sep_street_signs,
    'район':sep_district_signs,
    'регион':sep_region_signs,
    'страна':{'страна', 'стр'},
    'индекс':{'индекс', 'инд'}
  }
  
  return types

In [30]:
def load_cities():
  # https://maps.vlasenko.net/russia/ru-list.csv
  long_cities = pd.read_excel('cities.xlsx', header=None)
  long_cities.columns = ['region', 'district', 'town', 'x', 'y']
  cities_list = []
  for tokens in long_cities['town'].apply(tokenize):
    cities_list.extend(tokens)
  return set(cities_list)

In [2]:
def load_regions():
  regions_list = pd.read_excel('regions.xlsx')
  sep_regions_list = {}
  for line in regions_list.itertuples():
    try:
      sep_regions_list[line[3]].update({token for token in tokenize(line[2])})
    except KeyError:
      sep_regions_list[line[3]]={token for token in tokenize(line[2])}
  return sep_regions_list

In [3]:
def load_streets():
  Streets = pd.read_csv('streets.tsv', sep='\t')
  streetset=['парканат', 'вознесенское']
  for tokens in Streets['Парканат (Вознесенское)'].astype(str).apply(tokenize):
    streetset.extend(tokens)
  return set(streetset)

In [8]:
def load_names():
  names = {
    'улица':load_streets(),
    'город':load_cities(),
    'регион':load_regions(),
    'страна':{
      'россия':{"россия", "российская", "федерация", "рф"},
      'беларусь':{"белоруссия", "беларусь"},
      'украина':{"украина"}
    }
  }
  return names

In [389]:
def load_vocabulary(vocab = 'vocab.json'):
  import json
  try:
    with open(vocab, 'r') as fs:
      vocabulary = to_set(json.load(fs))
      return vocabulary
  except FileNotFoundError:
    pass
  
  vocabulary = {
    'типы': load_types(),
    'названия': load_names(),
    'препинания':{',', '-', '.', '/'}
  }
  
  ###
  ###    Теперь надо их очистить друг от друга
  ###
  
  # Города федерального значения считаются регионами
  for a in ['москва', 'санкт-петербург', 'севастополь']:
    try:
      vocabulary['названия']['город'].remove(a)
    except:
      pass
  
  # самые замусореные словари - это города и улиц, так как они черпаются из ФИАС
  for name in ['город', 'улица']:
    for that_type in ['улица', "дом", "район", "регион", "город", 'страна', 'индекс']:
      vocabulary['названия'][name] = diff_sets(vocabulary['названия'][name], whole_set(vocabulary['типы'][that_type]))

  vocabulary['названия']['улица'].remove('россия')
  vocabulary['названия']['улица'].remove('москва')
  vocabulary['названия']['улица'].update('б')
  
  return vocabulary

In [46]:
def save_vocab(vocabulary, name = 'vocab.json'):
  import json
  class SetEncoder(json.JSONEncoder):
    def default(self, obj):
      if isinstance(obj, set):
         return list(obj)
      return json.JSONEncoder.default(self, obj)
  with open(name, 'w') as fs:
    json.dump(vocabulary, fs, cls=SetEncoder)

In [385]:
def to_set(nested_dict):
  if isinstance(nested_dict, dict):
    new_dict = nested_dict.copy()
    for key, value in nested_dict.items():
       new_dict[key] = to_set(value)
    return new_dict
  elif isinstance(nested_dict, list):
    return set(nested_dict)
  elif isinstance(nested_dict, set):
    return nested_dict

In [335]:
%time vocabulary = load_vocabulary()

Удалено 4 записей
Удалено 4 записей
Удалено 2 записей
Удалено 2 записей
Удалено 3 записей
Удалено 1 записей
Удалено 0 записей
Удалено 19 записей
Удалено 16 записей
Удалено 9 записей
Удалено 12 записей
Удалено 5 записей
Удалено 1 записей
Удалено 2 записей
CPU times: user 44 s, sys: 511 ms, total: 44.5 s
Wall time: 45.2 s


In [390]:
save_vocab(vocabulary)

## Стандартизация адреса 

Стандартизация происходит по 20 полям как в ДаДате. Составим табличку поиска

In [59]:
LUT = {    # Look-up-table
  'типы':{
    'город':7,
    'дом':{
        'дом': 15,
        'корпус': 17,
        'квартира': 19,
        'офис': 19,
        'литера': 30},
    'улица':13,
    'район':5,
    'регион':3,
    'страна':32,
    'индекс':31
  },
  'названия':{
    'улица':14,
    'город':8,
    'страна':2,
    'район':4,
    'регион':4
  },
  'препинания':0
} 

In [61]:
def titles_():
  target_fields = ['Запятая', 'Индекс', 'Страна', 'Тип региона',
                 'Регион', 'Тип района', 'Район',
                 'Тип города', 'Город', 'Тип н/п',
                 'Н/п', 'Адм. округ', 'Район города',
                 'Тип улицы', 'Улица', 'Тип дома', 'Дом',
                 'Тип корпуса/строения', 'Корпус/строение',
                 'Тип квартиры', 'Номер Квартиры', 'Не распознано', 'число']
  dic = {}
  for i in range(len(target_fields)):
    dic[i] = target_fields[i]
  dic[30] = 'литера'
  dic[31] = 'обозначение индекса'
  dic[32] = 'обозначение страны'
  return dic

In [62]:
titles = titles_()

### Вспомогательные методы

In [53]:
def set_intersect(set1, set2):
  """
  Возвращает общие значения двух сетов
  """
  set3 = []
  for i in set1:
    if i in set2:
      set3.append(i)
  return set(set3)

In [307]:
def map_types(token):
  ''' Возвращает номер класса '''
  if token == ',':
    return [0]
  elif token.isdigit():
    if is_index(token): 
      return [1]
    elif len(str(token))>6:
      return [21]
    else: 
      return [22]
  else:
    possible_types = decode_dict(search_dict(token, vocabulary))
    if len(possible_types) == 1:
      return possible_types
    elif len(possible_types) > 1:
      return possible_types
    else:
      return [21]

### Самый главный и самый лагучий детектор

In [320]:
def clear_street(types):
  '''
  Определяется с названием улицы. 
  После того как определится — удаляет это городое звание со всех остальных ячеек, оставляя меньше вариантов.
  '''
  # находит последовательность типов, обзывает её улицей и говорит что все остальные это точно не улица
  signs = get_index(types, 13)
  streets = get_index(types, 14)
  signs.extend(streets)
  # самая длительная последовательность объявляется победителем
  curr_length = 0 
  max_length = 0
  items = []
  if signs == []:
    signs = streets
  
  for i in signs:
    # go right
    j = i
    curr_length = 0
    while j<len(types) and (14 in types[j] or 13 in types[j]):
      curr_length += 1
      if curr_length >= max_length:
        max_length = curr_length
        items = list(range(i,j+1))
      j += 1
    
    # go left
    j = i
    curr_length = 0
    while j>0 and (14 in types[j] or 13 in types[j]):
      curr_length += 1
      if curr_length >= max_length:
        max_length = curr_length
        items = list(range(j, i+1))
      j -= 1
  
  # теперь оставляем звание улицы только за выбранной последовательностью
  for i, element in enumerate(types):
    if 14 in element and i not in items:
      if len(element)>1:
        types[i].remove(14)
      else:
        types[i] = [21]
    elif 13 in element and i in items:
      types[i] = [13]
    elif 14 in element and i in items:
      types[i] = [14]
  return types

def is_index(string):
  if len(string)!=6:
    return False
  return string.isdigit()

def check_num(types, i):
  '''
  Этот детектор определяет что из чисел является номером дома, что номером квартиры итд
  '''
  if i-1>=0:
    if 0 in types[i-1] and i-2>0 and (14 in types[i-2] or 13 in types[i-2]):
      return [16]
    if 15 in types[i-1]:
      types[i-1] = [15]
      return [16]
    elif 17 in types[i-1]:
      types[i-1] = [17]
      return [18]
    elif 19 in types[i-1]:
      types[i-1] = [19]
      return [20]
    elif 14 in types[i-1]:
      types[i-1] = [14]
      return [16]
#     elif 13 in types[i-1]:
#       types[i-1] = [13]
#       return [14]
    else:
      return [14]
  else:
    return types[i]

In [340]:
def case37(types, i):
  '''
  исправляет г. Москва на регион Москва
  '''
  if (3 in types[i]) and (7 in types[i]):
    if i-1>0 and (4 in types[i-1]):
      types[i] = [3]
      types[i-1] = [4]
    elif i+1<len(types) and (8 in types[i+1]):
      types[i] = [7]
      types[i+1] = [8]
    elif i+1<len(types) and (4 in types[i+1]):
      types[i] = [3]
      types[i+1] = [4]
    elif i-1>0 and (8 in types[i-1]):
      types[i] = [7]
      types[i-1] = [8]
  return types

In [354]:
def case30(types, i):
  '''
  Если литера между номеров дома — это литера
  '''
  if 30 in types[i]:
    if i-1>0:
      for num in [16, 17, 18, 19, 20]:
        if num in types[i-1]:
          types[i] = [30]
          return types
    elif i+1<len(types):
      for num in [17, 18, 19, 20]:
        if num in types[i+1]:
          types[i] = [30]
          return types
  return types

In [342]:
special_cases = {
  30:case30,
  22:check_num,
  3:case37,
  7:case37 
}

In [143]:
def overlaps(values, target):
  '''
  Отвечает на вопрос:"Есть ли у двух сетов общие значения"
  '''
  if isinstance(values, list) or isinstance(values, set):
    for value in values:
      try:
        if value in target:
          return True
      except TypeError:
        if value == target:
          return True
    return False
  else:
    try:
      if values in target:
        return True
    except TypeError:
      if values == target:
        return True
    return False

def get_index(nested_list, element):
  response = []
  for i, array in enumerate(nested_list):
    try:
      if element in array:
        response.append(i)
    except TypeError:
      if element == array:
        response.append(i)
  return response

In [57]:
def decode_dict(nested_list):
  '''returns no. of type'''
  result = []
  for array in nested_list:
    temp = LUT
    for string in array:
      temp = temp[string]
      if isinstance(temp, int):
        result.append(temp)
        break
  return result

In [70]:
def search_dict(word, dic):
  '''
  Поиск по словарю. Ищет по всем вложенным рубрикам словаря и возвращает путь к найденному слову
  '''
  if isinstance(dic, dict):
    winners = []
    for key, value in dic.items():
      response = search_dict(word, value)
      if response == True:
        winners.append(key)
      elif isinstance(response, list):
        for instance in response:
          if isinstance(instance, list):
            winners.append([key, *instance])
          else:
            winners.append([key, instance])
      elif isinstance(response, str):
        winners.append([key, response])
    return winners
  elif isinstance(dic, set) or isinstance(dic, list):
    if word in dic:
      return True

### Выводы для главного метода

In [363]:
def output(tokens, types):
  for i, _ in enumerate(types):
    if isinstance(types[i], list):
      if types[i] != [0]:
        print(tokens[i], ' : ', [titles[types[i][j]] for j, _ in enumerate(types[i])])
    else:
      if types[i] != 0:
        print(tokens[i], ' : ', titles[types[i]])

In [373]:
def to_dict(tokens, types, original = True):
  address = {}
  if original:
    address['original'] = original
  for i, _ in enumerate(types):
    if isinstance(types[i], list):
      if types[i] != [0]:
        try:
          address[titles[types[i][0]]]+= (tokens[i]+ " ")
        except KeyError:
          address[titles[types[i][0]]] = (tokens[i]+ " ")
    else:
      if types[i] != 0:
        try:
          address[titles[types[i]]] += (tokens[i]+ " ")
        except KeyError:
          address[titles[types[i]]] = (tokens[i]+ " ")
  return address

### Тот самый метод

In [380]:
def standardize(string, out = False, original = True):
  '''
  In[]: string with address
  Out[]: dunno
  '''
  tokens = tokenize(string, comma=True)
  types_map = [map_types(x) for x in tokens]  
  for i, types in enumerate(types_map):
    if 22 in types_map[i]:
      types_map[i] = check_num(types_map, i)
  types_map = clear_street(types_map)
  for i, types in enumerate(types_map):  
    if len(types_map[i]) > 1:
      for option in types_map[i]:
        if option in special_cases.keys():
          types_map = special_cases[option](types_map, i)
  if out:
    output(tokens, types_map)
  if original:
    original = string
  return to_dict(tokens, types_map, original)

# Проверка работоспособности

In [290]:
ref_gz = pd.read_excel('ref/gz.xlsx')

In [309]:
order = ['Индекс','Страна','Тип региона','Регион','Тип города','Город', 'Тип района', 'Тип улицы', 'Улица', 'Тип дома', 'Дом', 'литера', 'Тип корпуса/строения', 'Корпус/строение', 'Тип квартиры', 'Номер Квартиры', 'original', 'Не распознано', 'обозначение страны', 'обозначение индекса']

In [324]:
pd.DataFrame(list(ref_gz['origin'].apply(standardize)), columns=order).to_excel('results/gz.xlsx')

In [384]:
pd.DataFrame(list(ref_gz['origin'].apply(standardize)), columns=order)

,Индекс,Страна,Тип региона,Регион,Тип города,Город,Тип района,Тип улицы,Улица,Тип дома,Дом,литера,Тип корпуса/строения,Корпус/строение,Тип квартиры,Номер Квартиры,original,Не распознано,обозначение страны,обозначение индекса
0,107113,NaN,г,москва,NaN,NaN,NaN,ул,лобачика,д,11,NaN,NaN,NaN,NaN,NaN,"107113, г. Москва, ул. Лобачика, д. 11",NaN,NaN,NaN
1,129301,российская федерация,г,москва,NaN,NaN,NaN,ул,бориса галушкина,д,17,NaN,NaN,NaN,NaN,NaN,"129301, Российская Федерация, г. Москва, Борис...",NaN,NaN,NaN
2,111394,NaN,г,москва,NaN,NaN,NaN,ул,перовская,д,65,NaN,стр,1,NaN,NaN,"111394, Москва г, ул.Перовская, \n\nд.65, стр....",NaN,NaN,NaN
3,143406,NaN,область,NaN,г,московская красногорск,NaN,ул,50 лет октября,д,12,NaN,NaN,NaN,NaN,NaN,"143406, Московская область, г. Красногорск, ул...",NaN,NaN,NaN
4,127051,NaN,г,москва,NaN,NaN,NaN,переулок,сухаревский м,д,9,а,стр,1,пом ком,1 56,"127051, г. Москва, переулок Сухаревский М.,д.9...",NaN,NaN,NaN
5,413850,NaN,обл,NaN,г,балаково,NaN,ул,коммунистическая,д,124,NaN,NaN,NaN,NaN,NaN,"413850, г. Балаково, Саратовской обл., ул. Ком...",саратовской,NaN,NaN
6,123995,NaN,г,москва,NaN,NaN,NaN,наб,бережковская,д,20,NaN,стр,9,комната,27,"123995, г. Москва, Бережковская наб., д.20, ст...",NaN,NaN,NaN
7,105094,NaN,г,москва,NaN,NaN,NaN,ул,гольяновская,д,6,NaN,NaN,NaN,NaN,NaN,"105094, г. Москва ул. Гольяновская, д. 6",NaN,NaN,NaN
8,664116,NaN,NaN,NaN,г,омск,NaN,ул,27 северная,д,48,NaN,NaN,NaN,офис,217,"664116, г. Омск, ул. 27 Северная, д.48, офис 2...",1,NaN,NaN
9,142000,NaN,NaN,NaN,г,домодедово,NaN,пр,кутузовский,д,18,NaN,NaN,NaN,кв ком,102 2,"142000, МО, г. Домодедово, Кутузовский пр. д.1...",мо,NaN,NaN
